In [2]:
! pip install sympy==1.13.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 17.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: sympy
    Found existing installation: sympy 1.12
    Uninstalling sympy-1.12:
      Successfully uninstalled sympy-1.12


In [3]:
def debug_print(message):
    print(f"{message}")

In [27]:
import sympy as sp
from sympy.logic.boolalg import And, Or, Not
from sympy.logic.inference import satisfiable
from src.tokenizer import Tokenizer
from src.parser import Parser
from src.simplifier import Simplifier
from src.config import debug_print

class Comparator:
   def __init__(self):
       self.tokenizer = Tokenizer()
       self.simplifier = Simplifier()

   def compare(self, predicate1: str, predicate2: str) -> str:
       # Tokenize, parse, and simplify the first predicate
       tokens1 = self.tokenizer.tokenize(predicate1)
       debug_print(f"Tokens1: {tokens1}")
       parser1 = Parser(tokens1)
       ast1 = parser1.parse()
       print(f"Parsed AST1: {ast1}")

       # Tokenize, parse, and simplify the second predicate
       tokens2 = self.tokenizer.tokenize(predicate2)
       debug_print(f"Tokens2: {tokens2}")
       parser2 = Parser(tokens2)
       ast2 = parser2.parse()
       print(f"Parsed AST2: {ast2}")

       # Convert ASTs to SymPy expressions
       expr1 = self._to_sympy_expr(ast1)
       expr2 = self._to_sympy_expr(ast2)

       # Simplify expressions
       print(f"SymPy Expression 1: {expr1}")
       simplified_expr1 = sp.simplify(expr1)
       print(f"Simplified SymPy Expression 1: {simplified_expr1}")

       print(f"SymPy Expression 2: {expr2}")
       simplified_expr2 = sp.simplify(expr2)
       print(f"Simplified SymPy Expression 2: {simplified_expr2}")

       # Manually check implications
       implies1_to_2 = self._implies(expr1, expr2)
       print(f"> Implies expr1 to expr2: {implies1_to_2}")
       implies2_to_1 = self._implies(expr2, expr1)
       print(f"> Implies expr2 to expr1: {implies2_to_1}")

       if implies1_to_2 and not implies2_to_1:
           return "The first predicate is stronger."
       elif implies2_to_1 and not implies1_to_2:
           return "The second predicate is stronger."
       elif implies1_to_2 and implies2_to_1:
           return "The predicates are equivalent."
       else:
           return "The predicates are not equivalent and neither is stronger."

   def _to_sympy_expr(self, ast):
       if not ast.children:
           try:
               # Try converting to int or float if the value is a numeric string
               value = float(ast.value) if '.' in ast.value else int(ast.value)
               return sp.Number(value)
           except ValueError:
               # If conversion fails, treat it as a symbol
               return sp.Symbol(ast.value.replace('.', '_'))
       args = [self._to_sympy_expr(child) for child in ast.children]
       if ast.value in ('&&', '||', '!', '==', '!=', '>', '<', '>=', '<='):
           return getattr(sp, self._sympy_operator(ast.value))(*args)
       elif ast.value == '/':
           return sp.Mul(sp.Pow(args[1], -1), args[0])
       elif ast.value == '+':
           return sp.Add(*args)
       elif ast.value == '-':
           return sp.Add(args[0], sp.Mul(-1, args[1]))
       elif ast.value == '*':
           return sp.Mul(*args)
       elif '()' in ast.value:
           func_name = ast.value.replace('()', '')
           return sp.Function(func_name)(*args)
       return sp.Symbol(ast.value.replace('.', '_'))

   def _sympy_operator(self, op):
       return {
           '&&': 'And',
           '||': 'Or',
           '!': 'Not',
           '==': 'Eq',
           '!=': 'Ne',
           '>': 'Gt',
           '<': 'Lt',
           '>=': 'Ge',
           '<=': 'Le'
       }[op]

   def _implies(self, expr1, expr2):
       """
       Check if expr1 implies expr2 by manually comparing the expressions.
       """
       debug_print(f"Checking implication: {expr1} -> {expr2}")
       if expr1 == expr2:
           debug_print("Expressions are identical.")
           return True

       # Handle AND expression for expr2
       if isinstance(expr2, And):
           # expr1 should imply all parts of expr2 if expr2 is an AND expression
           results = [self._implies(expr1, arg) for arg in expr2.args]
           debug_print(f"Implication results for And expr2 which was `{expr1} => {expr2}`: {results}")
           return all(results)
      
       # Handle AND expression for expr1
       if isinstance(expr1, And):
           # All parts of expr1 should imply expr2 if expr1 is an AND expression
           results = [self._implies(arg, expr2) for arg in expr1.args]
           debug_print(f"Implication results for And expr1 which was `{expr1} => {expr2}`: {results}")
           return any(results)

       # Handle OR expression for expr2
       if isinstance(expr2, Or):
           # expr1 should imply at least one part of expr2 if expr2 is an OR expression
           results = [self._implies(expr1, arg) for arg in expr2.args]
           debug_print(f"Implication results for Or expr2 which was `{expr1} => {expr2}`: {results}")
           return any(results)
      
       # Handle OR expression for expr1
       if isinstance(expr1, Or):
           # All parts of expr1 should imply expr2 if expr1 is an OR expression
           results = [self._implies(arg, expr2) for arg in expr1.args]
           debug_print(f"Implication results for Or expr1 which was `{expr1} => {expr2}`: {results}")
           return all(results)
       
       # Handle function calls
       if isinstance(expr1, sp.Function) and isinstance(expr2, sp.Function):
           # Ensure the function names and the number of arguments match
           if expr1.func == expr2.func and len(expr1.args) == len(expr2.args):
               return all(self._implies(arg1, arg2) for arg1, arg2 in zip(expr1.args, expr2.args))
           return False
       
       if isinstance(expr1, sp.Symbol) and isinstance(expr2, sp.Symbol):
           return expr1 == expr2

       # Specific relational operator checks for numerical comparisons
       relational_operators = (sp.Gt, sp.Ge, sp.Lt, sp.Le, sp.Eq, sp.Ne)
       if isinstance(expr1, relational_operators) and isinstance(expr2, relational_operators):
           print(f'we are here!... expr1: {expr1}, expr2: {expr2}')
           if all(isinstance(arg, (sp.Float, sp.Integer, sp.Symbol)) for arg in [expr1.lhs, expr1.rhs, expr2.lhs, expr2.rhs]):
               print(f'Inside!... expr1: {expr1}, expr2: {expr2}')
               # Check if the negation of the implication is not satisfiable
               negation = sp.And(expr1, Not(expr2))
               print(f"Negation of the implication {expr1} -> {expr2}: {satisfiable(negation)}; type of {type(satisfiable(negation))}")
               result = not satisfiable(negation, use_lra_theory=True)
               print(f"Implication {expr1} -> {expr2} using satisfiable: {result}")
               return result
      
       # Check if the negation of the implication is not satisfiable for general expressions
       print(f'Expression 1 is: {expr1}, and its type is {type(expr1)}')
       print(f'Expression 2 is: {expr2}, and its type is {type(expr2)}')
       negation = sp.And(expr1, Not(expr2))
       result = not satisfiable(negation, use_lra_theory=True)
       print(f"Implication {expr1} -> {expr2} using satisfiable: {result}")
       return result

# Example usage
# predicate1 = "_getIdentifierWhitelist().isIdentifierSupported(_priceIdentifier)"
# predicate2 = "_getIdentifierWhitelist().isIdentifierSupported(smt)"

# predicate1 = "(_tTotalpercentBuy)/divisorBuy>=(_tTotal/5000)" 
# predicate2 = "(percentBuy_decimals)/divisorBuy>=(_tTotal/10000)"

# predicate1 = "12 < a"
# predicate2 = "a < 12"



# predicate1 = "a < 12"
# predicate2 = "a < 13"

predicate1 = "a < 12"
predicate2 = "a == 12"


comparator = Comparator()
result = comparator.compare(predicate1, predicate2)
print(result)


Parsed AST1: ASTNode(value='<', children=[ASTNode(value='a', children=[]), ASTNode(value='12', children=[])])
Parsed AST2: ASTNode(value='==', children=[ASTNode(value='a', children=[]), ASTNode(value='12', children=[])])
SymPy Expression 1: a < 12
Simplified SymPy Expression 1: a < 12
SymPy Expression 2: Eq(a, 12)
Simplified SymPy Expression 2: Eq(a, 12)
we are here!... expr1: a < 12, expr2: Eq(a, 12)
Inside!... expr1: a < 12, expr2: Eq(a, 12)
Negation of the implication a < 12 -> Eq(a, 12): {Q.lt(a, 12): True, Q.ne(a, 12): True}; type of <class 'dict'>


AssertionError: 

In [3]:
import sympy as sp
from typing import Union
from src.parser import ASTNode
from src.config import debug_print

class Simplifier:
    def __init__(self):
        self.symbols = {
            'msg.sender': sp.Symbol('msg_sender'),
            'msg.origin': sp.Symbol('msg_origin'),
            '==': sp.Eq,
            '!=': sp.Ne,
            '>=': sp.Ge,
            '<=': sp.Le,
            '>': sp.Gt,
            '<': sp.Lt,
            '&&': sp.And,
            '||': sp.Or,
            '!': sp.Not
        }

    def simplify(self, ast: ASTNode) -> Union[str, ASTNode]:
        debug_print(f"Simplifying AST: {ast}")
        sympy_expr = self._to_sympy(ast)
        debug_print(f"Converted to sympy expression: {sympy_expr}")
        simplified_expr = sp.simplify(sympy_expr)
        debug_print(f"Simplified sympy expression: {simplified_expr}")
        simplified_ast = self._to_ast(simplified_expr)
        debug_print(f"Converted back to AST: {simplified_ast}")
        return simplified_ast

    def _to_sympy(self, node: ASTNode):
        if node.value in self.symbols and not node.children:
            return self.symbols[node.value]
        elif node.value in self.symbols:
            if node.value in ('&&', '||'):
                return self.symbols[node.value](*[self._to_sympy(child) for child in node.children])
            elif node.value == '!':
                return self.symbols[node.value](self._to_sympy(node.children[0]))
            elif len(node.children) == 2:
                return self.symbols[node.value](self._to_sympy(node.children[0]), self._to_sympy(node.children[1]))
            else:
                raise ValueError(f"Invalid number of children for operator {node.value}")
        elif isinstance(node.value, (int, float)):
            return sp.Number(node.value)
        else:
            # Preserve function calls and other identifiers as-is
            if '(' in node.value and ')' in node.value:
                func_name = node.value  # Ensure the function name is preserved entirely
                args = node.children
                return sp.Function(func_name)(*map(self._to_sympy, args))
            else:
                return sp.Symbol(node.value.replace('.', '_'))

    def _to_ast(self, expr):
        if isinstance(expr, sp.Equality):
            return ASTNode('==', [self._to_ast(expr.lhs), self._to_ast(expr.rhs)])
        elif isinstance(expr, sp.Rel):
            op_map = {'>': '>', '<': '<', '>=': '>=', '<=': '<=', '!=': '!='}
            return ASTNode(op_map[expr.rel_op], [self._to_ast(expr.lhs), self._to_ast(expr.rhs)])
        elif isinstance(expr, sp.And):
            return ASTNode('&&', [self._to_ast(arg) for arg in expr.args])
        elif isinstance(expr, sp.Or):
            return ASTNode('||', [self._to_ast(arg) for arg in expr.args])
        elif isinstance(expr, sp.Not):
            return ASTNode('!', [self._to_ast(expr.args[0])])
        elif isinstance(expr, sp.Function):
            func_name = str(expr.func)
            return ASTNode(func_name, [self._to_ast(arg) for arg in expr.args])
        else:
            return ASTNode(str(expr))


ModuleNotFoundError: No module named 'sympy'

In [98]:

tokenizer = Tokenizer()
simplifier = Simplifier()

tokens1 = tokenizer.tokenize(predicate1)
parser1 = Parser(tokens1)
ast1 = parser1.parse()
simplified_ast1 = simplifier.simplify(ast1)

print(f"predicate1: {predicate1}")
print(f"AST1 is: {ast1}")
print(f"Simplified AST1: {simplified_ast1}")


print('--------------------------------------------------------------------------------------------')

tokens2 = tokenizer.tokenize(predicate2)
parser2 = Parser(tokens2)
ast2 = parser2.parse()
simplified_ast2 = simplifier.simplify(ast2)

print(f"predicate1: {predicate2}")
print(f"AST2 is: {ast2}")
print(f"Simplified AST1: {simplified_ast2}")


predicate1: _getIdentifierWhitelist().isIdentifierSupported(_priceIdentifier)
AST1 is: ASTNode(value='_getIdentifierWhitelist().isIdentifierSupported()', children=[ASTNode(value='_priceIdentifier', children=[])])
Simplified AST1: ASTNode(value='_getIdentifierWhitelist().isIdentifierSupported()', children=[ASTNode(value='_priceIdentifier', children=[])])
--------------------------------------------------------------------------------------------
predicate1: _getIdentifierWhitelist().isIdentifierSupported(smt)
AST2 is: ASTNode(value='_getIdentifierWhitelist().isIdentifierSupported()', children=[ASTNode(value='smt', children=[])])
Simplified AST1: ASTNode(value='_getIdentifierWhitelist().isIdentifierSupported()', children=[ASTNode(value='smt', children=[])])


In [24]:
predicate1, predicate2 = "(_tTotalpercentBuy)/divisorBuy>=(_tTotal/5000)", "(percentBuy_decimals)/divisorBuy>=(_tTotal/10000)"
comparator = Comparator()
result = comparator.compare(predicate1, predicate2)
print(result)

The predicates are equivalent.
